<a href="https://colab.research.google.com/github/placerda/region-growing/blob/master/region_growing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Parte 1: Configuração

Importa bibliotecas necessárias e configura parâmetros de execução. 

**Parâmetros**

```
DICOM_FOLDER = diretório com os arquivos DICOM
RESULT_FOLDER = diretório onde será gerado o resultado da segmentação
SEMENTES = lista de sementes inseridas manualmente, exemplo: [[183, 231, 310]. A sequência é: z, y, x.
LIMIAR = limiar  a ser utilizado para inclusão dos voxels na imagem a partir da distância entre os vetores de features.
```




In [0]:
# monta google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# instala e importa as bibliotecas
!pip install pycuda pydicom
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

# se necessário incluir módulos python customizados utilizar o trecho abaixo
# import sys 
# sys.path.append(LIB_FOLDER)

# importa os módulos utilizados nas próximas células
import numpy as np
import pydicom
import os
import pycuda.autoinit 
from pycuda import gpuarray
from pycuda.compiler import SourceModule
import cv2


In [0]:
# configura parâmetros
MAX_CORTES = 448
DICOM_FOLDER = '/content/gdrive/My Drive/uff/region_growing/dataset/Imacx01_Animal1/dicom'
RESULT_FOLDER = '/content/gdrive/My Drive/uff/region_growing/results/Imacx01_Animal1/segmentation'
WIDTH = 512
HEIGHT = 512
# define as sementes 
SEMENTES = [[151, 220, 218]] # [z, y, x]
LIMIAR = 0.2

## Parte 2: Aquisição das Imagens

Carrega os dados da imagem dicom na memória principal e copia na memória do device.

In [0]:
# carrega os dados da imagem e das sementes na memória da CPU
files = os.listdir(DICOM_FOLDER)
files.sort()
num_cortes = len(files) if len(files) < MAX_CORTES else MAX_CORTES
# matriz 3D com os valores HU
h_image_data = np.zeros((num_cortes, HEIGHT, WIDTH), dtype='int32')

# carrega os cortes na memória principal (CPU)
print ('carregando cortes 1 a {}.\n'.format(num_cortes))
for idx, file in enumerate(files):
    # le arquivos dicom
    if (idx < num_cortes):
        filename = "{}/{}".format(DICOM_FOLDER, os.fsdecode(file))
        
        ds = pydicom.dcmread(filename)
        b = ds.RescaleIntercept
        m = ds.RescaleSlope
        slice = m * ds.pixel_array + b
        slice = np.int32(slice) # define como inteiro 32-bit para usar int no kernel cuda
        h_image_data[idx] = slice

        # feedback de processamento para o usuário
        print('.', end='')
        if ((idx+1) % 100 == 0): print('') 
        
print ('\nfim da carga dos slices na memória principal.')

# copia para a memória do device (GPU)
d_image_data = gpuarray.to_gpu(h_image_data)
print ('fim da cópia para memória global.')

## Parte 3: Crescimento de Região 3D em Paralelo

Executa o loop de crecimento de região até que não exista mais nenhum pixel a ser incluído.


In [0]:
# vetor com as sementes
h_seed_data = np.array(SEMENTES, dtype='int32')

# matriz 3D com a região onde serão incluidas as sementes
# cada semente na matriz indica o equivalente a sua ordem no vetor de sementes (iniciando em 1)
h_region_data = np.zeros((num_cortes, HEIGHT, WIDTH), dtype='int32')

# inicializa a matriz 3D com a posição das sementes
for idx, semente in enumerate(h_seed_data):    
    h_region_data[semente[0], semente[1], semente[2]] = idx+1

d_region_data = gpuarray.to_gpu(h_region_data)
d_seed_data = gpuarray.to_gpu(h_seed_data)


# define o kernel
ker = SourceModule("""

#define _LIMIAR 0.3
#define _NUM_FEATURES 4
#define _MIN_HU -1024
#define _MAX_HU 1024

#define _X  ( threadIdx.x + blockIdx.x * blockDim.x )
#define _Y  ( threadIdx.y + blockIdx.y * blockDim.y )
#define _Z  ( threadIdx.z + blockIdx.z * blockDim.z )
#define _WIDTH  512
#define _HEIGHT 512
#define _DEPTH  ( gridDim.z )

// antes de chamar essa função testar se as coordenadas estão dentro da figura
__device__ int get_index(int x, int y, int z){
  return x  + y * _WIDTH + z * _WIDTH * _HEIGHT;
}

// funcao que verifica se o voxel é vizinho da região, se verdadeiro retorna 
// o valor do elemento vizinho que corresponde a semente relacionada ao vizinho
__device__ int get_neighbor_seed(int x, int y, int z, int *in)
{
    int idx = get_index(x,y,z);
    for (int k = z-1; k <= z + 1; k++){
        for (int j = y-1; j <= y + 1; j++){
            for (int i = x-1; i <= x + 1; i++){
                // testa se está dentro da imagem
                if (((k > 0) && (k < _DEPTH)) && ((j > 0) && (j < _HEIGHT)) && ((i > 0) && (i < _WIDTH))){
                    int idx_neighbor = get_index(i, j, k);
                    if (idx_neighbor != idx)  // testa se não é o próprio elemento
                        if (in[idx_neighbor] > 0)  // se um dos vizinhos é > 0 ele é vizinho da região
                            return in[idx_neighbor];
                }
            }
        }
    }  
    return 0;
}

__device__ float normalizeHU(int hu){
  // faz um clip para excluir valores discrepantes
  if (hu < _MIN_HU) 
    hu = _MIN_HU;
  else if (hu > _MAX_HU){
    hu = _MAX_HU;
  } 
  return ((float)abs(hu-_MIN_HU))/abs(_MAX_HU-_MIN_HU);
}

// funcao para calcular o vetor de caracteristicas (HU, MEAN, MIN, MAX, CVE)
__device__ int calculate_features(int x, int y, int z, int *image, float *vector){  
  vector[0] = normalizeHU(image[get_index(x, y, z)]); //HU
  vector[1] = 0; // MEAN
  vector[2] = 0; // MIN
  vector[3] = 0; // MAX

  float min  = 1;
  float max  = 0;
  float sum  = 0;
  float qtde = 0;
  
  // calculates: mean, min and max
  for (int k = z-1; k <= z + 1; k++){
    for (int j = y-1; j <= y + 1; j++){
      for (int i = x-1; i <= x + 1; i++){
          if (((k > 0) && (k < _DEPTH)) && ((j > 0) && (j < _HEIGHT)) && ((i > 0) && (i < _WIDTH))){
            float hu = normalizeHU(image[get_index(i, j, k)]);
            sum+=hu;
            if (hu < min) min = hu;
            if (hu > max) max = hu;
            qtde++;
          }
      }
    }
  }
  vector[1] = (sum/qtde); // MEAN
  vector[2] = min; // MIN
  vector[3] = max; // MAX  
  return 0;
}

__device__ float calcula_distancia(float *vector1, float *vector2){
  float sum = 0;
  for (int i = 0; i < _NUM_FEATURES; i++){
    sum += pow((vector1[i] - vector2[i]), 2);
  }
  return (float)sqrt(sum);
}

__global__ void region_growing(int *image, int *region, int *seeds, int *incluidos, float limiar, float *debug)
{

   // x, y, z corresponde ao voxel executado por essa thread
   int x = _X, y = _Y, z = _Z;
   if ((x < _WIDTH) && (y < _HEIGHT) && (z < _DEPTH)){
    int idx = get_index(x,y,z);
    if (region[idx] == 0) { // só executa se ele ainda não entrou na região
         // verifica se o elemento é vizinho da região.
         int neighbor_seed = get_neighbor_seed(x, y, z, region);
         if (neighbor_seed > 0){
            // obtem as coordenadas da semente
            int coord_offset = 3; // coord: [z, y, x]
            int idx_seed = neighbor_seed - 1; // decrementa 1 pois índice do vetor começa em 0
            int seed_z = seeds[0+(idx_seed * coord_offset)];
            int seed_y = seeds[1+(idx_seed * coord_offset)];
            int seed_x = seeds[2+(idx_seed * coord_offset)];

            // obtem o vetor de features do elemento
            float vector1[_NUM_FEATURES];
            calculate_features(x, y, z, image, vector1);

            // obtem o vetor de features da semente
            float vector2[_NUM_FEATURES];
            calculate_features(seed_x, seed_y, seed_z, image, vector2);

            // calcula a distancia
            float distancia = calcula_distancia(vector1, vector2);

            // DEBUG
            // if ((x == 310) && (y == 231) && (z == 182)){
            //   debug[0] = distancia;
            // }
            // distancia = 10;
            // DEBUG

            if (distancia < limiar){
                region[idx] = neighbor_seed;
                incluidos[0] += 1;
            }

         }
    }

   }

}
""")

region_growing = ker.get_function("region_growing")

incluiu = True
conta_loops = 0
while (incluiu):
    # define e inicializa variaveis de controle
    h_incluidos = np.zeros((1), dtype='int32')
    d_incluidos = gpuarray.to_gpu(h_incluidos)
    h_debug = np.zeros((1), dtype='float32')    
    d_debug = gpuarray.to_gpu(h_debug)
    limiar = np.array(LIMIAR, dtype='float32')

    # executa o crescimento de região (obs: cada grid corresponde a um corte, por isso a dimensão z do grid é o número de cortes)
    region_growing(d_image_data, d_region_data, d_seed_data, d_incluidos, limiar, d_debug, grid=(16, 16, num_cortes), block=(32,32,1))
    pycuda.autoinit.context.synchronize()
    
    # atualiza variáveis no host
    h_incluidos = d_incluidos.get()
    incluiu = (h_incluidos[0] > 0) # verifica se houve inclusao

    # variavel utilizada para debug apenas
    h_debug = d_debug.get()    
    # print ('debug={}'.format(h_debug))
        
    # feedback do processamento para o usuário
    print('.', end='')
    conta_loops += 1
    if (conta_loops % 100 == 0): print('')

# copia o resultado na memória principal
h_segmented_data= d_region_data.get()
h_segmented_data[h_segmented_data>0] = 1
print('\nvolume da região segmentada: {}'.format(np.sum(h_segmented_data)))


## Parte 4: Grava em disco as imagens indicando a área segmentada

Faz o pós-processamento, gerando as imagens com a região segmentada e salvando
 o resultado em disco.

In [0]:
# configuração de janela para fazer a quantização em 256 cores
WINDOW_LENGHT=-100
WINDOW_WIDTH=1220
MIN_HU=-1024
MAX_HU=1024

# define intervalo para fazer o processamento
INITIAL_SLICE = 40 # 40
FINAL_SLICE = 200 # 240

# cria diretorio de saída se ele não existe
os.makedirs(RESULT_FOLDER, exist_ok=True)

# salva a matriz com a segmentação
np.save('{}/segmented_data.npy'.format(RESULT_FOLDER), h_segmented_data)

for idx, slice in enumerate(h_image_data):

    if (idx+1 >= INITIAL_SLICE) and (idx < FINAL_SLICE):

        slice_number = str(idx+1).zfill(4)

        # feedback do processamento para o usuário
        print ('{} '.format(slice_number), end='')
        if ((idx+2-INITIAL_SLICE) % 40 == 0): print ('')
        
        # quantização em tons de cinza 
        min_value = WINDOW_LENGHT - (WINDOW_WIDTH // 2)
        if min_value < MIN_HU: min_value = MIN_HU

        max_value = WINDOW_LENGHT + (WINDOW_WIDTH // 2)
        if max_value > MAX_HU: max_value = MAX_HU

        quantized = slice.copy()

        quantized = np.clip(quantized, min_value, max_value)

        # normaliza em tons de cinza
        for cell in np.nditer(quantized, op_flags=['readwrite']):
            cell[...] = ((cell - min_value) * 255) // (max_value - min_value)
        gray = quantized.astype('uint8')

        # aplica a mascara
        mask = h_segmented_data[idx]
        alpha = 0.6
        masked_image = np.dstack((gray, gray, gray))
        overlay = masked_image.copy()
        overlay[mask > 0] =  (0,255,0)
        cv2.addWeighted(overlay, alpha, masked_image, 1 - alpha, 0, masked_image)

        # calcula a area
        area_regiao = np.sum(mask)

        # salva arquivo no disco
        # inclui texto com a área
        cv2.putText(masked_image, "region area={}".format(area_regiao), (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), lineType=cv2.LINE_AA)

        # inclui texto com o número do slide
        cv2.putText(masked_image, "{}".format(slice_number), (225, 30), cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), lineType=cv2.LINE_AA)

        # salva imagem como jpg
        output_filepath = "{}/{}-auto.jpg".format(RESULT_FOLDER, slice_number)
        cv2.imwrite(output_filepath, masked_image)

Done!